# Project 1: Molecular Dynamics with OpenMP

This assignment is due in two weeks time, by **9:30 am on Thursday October 3rd**.

**You may work in pairs on this assignment:** When you officially submit this project on Canvas, you should indicate in the text submission field on Canvas:

- Who, if anyone you are working with
- If you are working in pairs, indicater whether the repository to be graded is yours or your partner's.
- Which commit of your repository you would like to be graded (we will grade the `master` branch by default if no choice is made)

**Which type of node are you using?** Because OpenMP can be used to program the GPUs, you may choose to optimize the application for any of the three types of nodes used in this class.  Declare the type of node you would like to use here:

**This notebook should be run on a node with 28 CPU cores and No \_ Tesla \_\_\_\_ GPUs**

If you will not use the GPUs, you should use the following modules:

In [1]:
module use $CSE6230_DIR/modulefiles
module load cse6230/core

|                                                                         |
|       A note about python/3.6:                                          |
|       PACE is lacking the staff to install all of the python 3          |
|       modules, but we do maintain an anaconda distribution for          |
|       both python 2 and python 3. As conda significantly reduces        |
|       the overhead with package management, we would much prefer        |
|       to maintain python 3 through anaconda.                            |
|                                                                         |
|       All pace installed modules are visible via the module avail       |
|       command.                                                          |
|                                                                         |


If you will use the GPUs, you should use the following modules:

In [ ]:
# module use $CSE6230_DIR/modulefiles
# module load cse6230/gcc-omp-gpu

(I've included a set of makefile rules for GNU-based builds: you can use `make MAKERULES=gcc` wherever you would use make and it should work.  You should do this if you are using the `gcc-omp-gpu` module or if you are developing on your laptop and don't have the intel compilers)

## About this program

The code for this assignment started out almost exactly the same as your third assignment with interacting particles.  We saw in that assignment the way that $O(n^2)$ interactions in an $n$-body simulation dominate the rest of the operations.  This project shows an attempt to return that work complexity from $O(n^2)$ back down to $O(n)$ or thereabouts.
  
Some of the potentials that define interactions in molecular dynamics decay *quite* rapidly.  So rapidly, that it is not a terrible approximation to assign to each particle an effective **radius $r$**.  If two particles are not touching (that is if their centers are more than $2r$ apart), then the interactions can safely be ignored (particularly if it will be drowned out relative to the background *Brownian* noise that we saw last week).  In side of $2r$, then the overlapping particles start pushing each other apart.

If you'd like to see the particulars of this assignments force due to interactions, you can look at `steric.h`, so called because the force approximate [steric effects](https://en.wikipedia.org/wiki/Steric_effects).

In [ ]:
# pygmentize steric.h

(If you find part of your program is compute bound, you are welcome to change the implementations in `steric.h`, as long as your still calculate the same function)

Now, suppose that our particles bounce around and repel each other until they are roughly in equilibrium.  We would expect that they would be well spread out, and that the chance of any two particles interacting would be no more likely than two particles placed at random.

A particle interacts with any particle within a range of $2*r$, which means that around each particle there is a sphere with volume $V_p = \frac{4}{3}\pi (2r)^3\approx 33 r^3$: any particle whose center is outside of that cell does not interact.  Suppose the volume of the periodic domain is $V_D$, and there are $N_p$ particles.  Then if the other $N_p - 1$ particles are distributed at random, then we expect $V_p (N_p - 1)/ V_D$ of those particles to interact with the particle in question.  Therefore we might expect $N_p V_p (N_p - 1) / 2 V_D$ interactions in total.

What's the point of this calculation?  Well, when run a periodic simulation, we are trying to approximate a larger domain with a fixed *density* of particles per volume.  Thus, if we consider $\phi = N_p/ V_D$ to be a fixed density of the problem we are trying to simulate, then the number of interactions is $\approx (N_p - 1) \phi / 2$.
*We should expect the number of interactions to scale linearly with the number of particles if we keep $\phi$ fixed.*

So how can we exploit the fact that only $O(N_p)$ interactions are expected instead of $O(N_p^2)$?  In our acceleration routine, we should try to rule out particles from interacting with each other.

One way to do this is *binning*: we divide up our periodic domain $[-L/2,L/2)^3$ into a grid of $b$ boxes per dimension, $b^3$ boxes total.  An algorithm would look like the following:

1. Given each particles coordinates, assign it to the appropriate box.
2. If the length of a box $(L / b)$ is longer than $2r$, then every particle can only interact with particles
  - In its own box,
  - In neighboring boxes
3. So loop over neighboring boxes and create a list of *pairs of particles* that are close enough to interact.

This is what is done now in `accelerate.c`: there is an interaction "object" that handles the internals of binning particles into boxes: it returns a list of pairs on request.

The previous $O(N_p^2)$ calculation is available for comparison and debugging purposes.

In [ ]:
# sed -n '54,85 p' accelerate.c | pygmentize -l c

## Your task

You're free to make just about any changes you'd like to the code.  The `cloud` program is currently a functioning serial program with a small amount of OpenMP already mixed in.  Below is a sequence of problems of increasing size $N_p$ but fixed density.

You should specify OpenMP environment variables before this loop that will be used by the programs.

In [ ]:
export OMP_NUM_THREADS=8
export OMP_PROC_BIND=close

In [1]:
for N_p in 1 2 4 8 16 32; do
  this_L=`echo "$N_p 0.333 20." | awk '{ print ($3 * $1^$2); }'`
  this_T=`echo "$N_p 25600" | awk '{ print ($2 / ($1 * $1)); }'`
  make runcloud NP=$(( 256*$N_p )) L=$this_L NT=$this_T PERF="perf stat"
done

make --silent clean
make cloud #--silent cloud
make[1]: Entering directory `/nv/coc-ice/bzhang376/cse6230/projects/1-molecular-dynamics'
icc -std=c99 -g -Wall -fPIC -O3 -xHost -qopt-report=5 -DMY_REMAINDER=1 -DMULTIVECTOR -DUPDATEINTERVAL=6 -I../../utils -I../../utils/tictoc  -openmp -c -o verlet.o verlet.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -std=c99 -g -Wall -fPIC -O3 -xHost -qopt-report=5 -DMY_REMAINDER=1 -DMULTIVECTOR -DUPDATEINTERVAL=6 -I../../utils -I../../utils/tictoc  -openmp -c -o accelerate.o accelerate.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -std=c99 -g -Wall -fPIC -O3 -xHost -qopt-report=5 -DMY_REMAINDER=1 -DMULTIVECTOR -DUPDATEINTERVAL=6 -I../../utils -I../../utils/tictoc  -openmp -c -o cloud.o cloud.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -std=c99 -g -Wall -fPIC -O3 -xHost -qopt-report=5

icc -std=c99 -g -Wall -fPIC -O3 -xHost -qopt-report=5 -DMY_REMAINDER=1 -DMULTIVECTOR -DUPDATEINTERVAL=6 -I../../utils -I../../utils/tictoc  -openmp -c -o vector.o vector.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -std=c99 -g -Wall -fPIC -O3 -xHost -qopt-report=5 -DMY_REMAINDER=1 -DMULTIVECTOR -DUPDATEINTERVAL=6 -I../../utils -I../../utils/tictoc  -openmp -c -o cloud_util.o cloud_util.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -std=c99 -g -Wall -fPIC -O3 -xHost -qopt-report=5 -DMY_REMAINDER=1 -DMULTIVECTOR -DUPDATEINTERVAL=6 -I../../utils -I../../utils/tictoc  -openmp -c -o interactions.o interactions.c
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc -openmp -o cloud verlet.o accelerate.o cloud.o initialize.o vector.o cloud_util.o interactions.o -lm -lrt
make[1]: Leaving directory `/nv/coc-ice/bzhang376/cse6230/projects

The interaction volume is 33.5103, so we expect 1.07531 interactions per particle, 2202.23 overall.

 Performance counter stats for './cloud 4096 100 1.e-4 100. 1. 50.3503 1.':

        108.267709      task-clock (msec)         #    0.984 CPUs utilized          
                30      context-switches          #    0.277 K/sec                  
                 0      cpu-migrations            #    0.000 K/sec                  
               715      page-faults               #    0.007 M/sec                  
       353,547,152      cycles                    #    3.265 GHz                    
   <not supported>      stalled-cycles-frontend  
   <not supported>      stalled-cycles-backend   
       521,703,109      instructions              #    1.48  insns per cycle        
        31,942,688      branches                  #  295.034 M/sec                  
         1,083,165      branch-misses             #    3.39% of all branches        

       0.110015686 seconds time elapsed



However, you code must still be correct:  an effective diffusion coefficient can be computed for the type of particles you are simulating.  The following diffusion coefficient calculation should stay in the range of 0.77-0.92:

In [2]:
make checkcloud NP=512 L=25.198421 NT=51000 CHUNK=1000

make --silent clean
make --silent cloud
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
icc: remark #10397: optimization reports are generated in *.optrpt files in the output location
./cloud 512 51000 1.e-4 100. 1. 25.198421 1. 1000 check | python3 check.py
Diffusion constant: [ 0.83613632]


## Grading

### 4 pts: Hassle-free usage: if the bash script that is generated by `jupyter convert` from this notebook runs without issue

### 6 pts: For code that correctly parallelizes all critical kernels (including the binning calculations in `interactions.c`)
 
- A correct diffusion coefficient is required for correctness
- If your code is not correct, points can be salvaged with *legible code* that describes what changes you are making

### 6 pts: Speed.  Any (correct) code that is the fastest on one of the benchmark problem sizes (for the node type you have selected) automatically gets 6 pts.  Code that does not outperform the initial version on any benchmark gets no points.  1 point will be available for each benchmark problem that shows non-trivial improvements in performance.

**Significant improvement is defined by the following speedups for the benchmark problems:**

- `N_p = 1`: 2x
- `N_p = 2`: 5x
- `N_p = 4, 8, 16, 32`: 9x

### 4 pts: Report.  In a cell below this one, describe the optimizations that you made and why you made them.

- Full points will require evidence (such as a screenshot) from `hpctoolkit` or some other profiling utility that motivates or justifies your changes.
- Points will be awarded for optimizations that you tried that did not work as long as you have a good explanation for why you tried them and why they didn't work.

1. ##### Change the size of boxs into an optimized one:

   If we only consider the time consumed by the step of finding interaction pairs, the best size should be the SMALLEST box size which fulfills the requirement that particles inside one box only interact with particles in the same box and in the nearby boxes. 

   Since the "radius" of the kind of particles is 1, based on the geometrical relationship, the optimized size of boxes should be $2\times2\times2$. So we changed the coefficient `boxdim` as

    $$\frac{20}{2}\;=\;10$$

   However, if the number of boxes is larger, there will also be extra time spent on the box-creating process. Since there is a larger array for labeling and saving boxes, the time spent on accessing the particles inside a box will also be longer.
   
   After the trial, we decided to set the `boxdim` as $8$.

![hpcviewer screen shot of GetPairs vs force](needOptGetPairs.png)

2. ##### Parallel the loop of finding interaction pairs: 

   Wee looked at profile of the program, and the GetPairs loop takes surprisingly large amount of time. To optimize, it must be parallelized. Calculation in each box is independent, which can be paralleled naturally. The most difficult part here is the data structure.

   In original program, all interaction pairs are recorded on one list. To parallel the process of finding and recording interaction pairs, the potential conflict of several threads writing on the list at the same time should be diminished.
   
   Firstly, an array of pair lists was built in `IXCreate`. The number of lists in the array is the number of threads. 
   
   Secondly, the finding process are executed on multiple threads (`IXGetPairs`). Every time one thread find an interacting pair, it push the pair onto the list it owns (`IXPushpair`).
   
   Finally, all the lists in the array are hoined into one list at the end of `IXGetPairs`, which will be sent to `accelerate`.

3. ##### Change the frequency of updating interaction pairs:
    
    In the original sequential program, the linked list of interaction pairs is updated in every time step. However, if the average speed of particles is not as high as flying over the distance as the diameter of the particle in a time step, it is not necessary to update it so frequently: the interaction pairs can keep interact in more than one time step, the two particles without interaction are also not so easy to correlate in a time step.
    
    The structure of `verlet_step` and `accelerate` were changed to make them more flexible on modifying the period or frequency of updating interaction pairs.
    
    We increase the interval to $6$ time steps and the diffusion constant calculated is still pretty close to the original, so we use this number.

4. ##### Parallel the function of computing the forces:
    
   After finding the interacting pairs, the force between one pair is only related to the positions of the two particles, which is independent with other pairs. So the loop of computing forces are paralleled. 

   However, since there is the risk of adding forces exerted on one particle simultaneously, the code of adding forces onto the elements of the force array "U" are set atomic. Everytime only one thread can access to the position saving the force on a particle.

5. ##### Replace the function `REMAINDER` by using `omp simd`: 
   
   A new function of computing the remainder of a number with regard to the dimension of the cell is written as `MY_REMAINDER` in the file "steric.h". The function is set under "omp declare simd", which let the code vecotrized better.

6. ##### Not using AoS: 
   
   Both the structure of saving the positions of particles AoS and SA in `vector.c` were tried. Using SoA is quicker than AoS. The reason of such difference may be that SoA is better for vectorization.

## Advice

- **My experience in the past:** Detailed git histories are correlated with better performance!
- **Understand your code before you try to change it:**
    - In addition to profiling utilities, it might be useful to add timers to
      individual routines.  The division of the program into objects that control
      different aspects of the code should make easy to, say, add a timer
      in one place without changing the whole program.
- **Simple problem parameters that can be changed:**
    - The number of boxes per dimension
    - The layout of vectors (array-of-structures or structure of arrays? see `vector.h`)
    - The data structures used to assign particles to boxes (is a linked-list really best)?
- **Avoid memory and other resource contention:**
    - Anytime multiple threads are trying to write to locations close to each
      other, it makes it difficult and expensive to make sure each thread has
      an up-to-date copy of the memory that is changing.  This would happen,
      for example, if many threads are writing to the `pairs` list in
      the interactions routine.  Consider allocating a separate workspace for
      each thread by, for example, giving each thread its own `pairs` array.
      Then, once all threads are done computing their pairs, you can combine
      the separate arrays into one array, or even change the interface of the
      `interactions()` function so that it is multiple lists are returned.
- **Find ways to avoid recomputing from scratch:**
    - Can you use the layout of the particles from the last time step to help you
      bin or find pairs in the next time step?
- **You get to choose how many threads we use to evaluate your code:**
    - There's nothing inherently wrong with achieving your best performance
      using fewer than the maximum number of threads available on a node.  The
      problem may simply not have enough concurrency to support every thread.
- **Read through these performance slides for ideas:** [From Archer](https://www.archer.ac.uk/training/course-material/2015/12/ShMem_OpenMP_York/Slides/L09-performance.pdf)
- **Reread the molecular dynamic notes from Prof. Chow to make sure you understand what we're trying to accomplish:** [Molecular dynamics and cell lists](https://www.cc.gatech.edu/%7Eechow/ipcc/hpc-course/05_celllist.pdf)